In [5]:
import numpy as np
import rsatoolbox
from rsatoolbox.util.matrix import pairwise_contrast
from scipy.spatial.distance import squareform
from scipy import io
import matplotlib.pyplot as plt
import rsatoolbox.data as rsd # abbreviation to deal with dataset
import rsatoolbox.rdm as rsr
from scipy import linalg
from scipy.optimize import minimize

In [8]:
matlab_data = io.matlab.loadmat('rdms_inferring/modelRDMs_A2020.mat')
matlab_data = matlab_data['modelRDMs']
n_models = len(matlab_data[0])
model_names = [matlab_data[0][i][0][0] for i in range(n_models)]
measurement_model = [matlab_data[0][i][1][0] for i in range(n_models)]
rdms_array = np.array([matlab_data[0][i][3][0] for i in range(n_models)])

In [9]:
model_rdms = rsatoolbox.rdm.RDMs(rdms_array,
                            rdm_descriptors={'brain_computational_model':model_names,
                                             'measurement_model':measurement_model},
                            dissimilarity_measure='Euclidean'
                           )

In [10]:
conv1_rdms = model_rdms.subset('brain_computational_model','conv1')
conv2_rdms = model_rdms.subset('brain_computational_model','conv2')

#fig, ax, ret_val = rsatoolbox.vis.show_rdm(conv1_rdms, rdm_descriptor='measurement_model', figsize=(10,10))

# fun = lambda a: np.sqrt((np.log(linalg.eigvalsh(G1,np.exp(a)*G2))**2).sum())
# res = minimize(fun,0, method='Nelder-Mead')
# print(np.exp(res.x))
# a_true = 1/np.exp(-np.log(linalg.eigvalsh(G1,G2)).sum()/np.shape(G1)[0])
# print(a_true)

In [1]:
!echo $PYTHONPATH

:/Users/mahdiyarshahbazi/Documents/GitHub/rsatoolbox


In [12]:
rsatoolbox.rdm.compare_neg_riemannian_distance(conv1_rdms,conv2_rdms)

AttributeError: module 'rsatoolbox.rdm' has no attribute 'compare_neg_riemannian_distance'